In [2]:
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.user import User
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.ad import Ad
from facebook_business.adobjects.adimage import AdImage
from facebook_business.adobjects.adcreative import AdCreative
from tqdm import tqdm_notebook
import datetime
import pandas as pd

# 페이스북 개발자 페이지에서 비지니스 앱 생성후 
# 마케팅 api 추가하여 권한 설정후 토큰 발급 받기
access_token = 'EAAC6R52Q8wIBAFZA5UbOv9YbyC6LttNt0NUbE5aohJbKq6MhaffZAWjHiWYAbsNRZBjEnE36Y52vl1OZCI3No6kCqiZAqoJWfHaoVztZBQvovHDtlMOxDBd3LViA6eBiKyZAZCr4QE8iZC2LdCDZAMdmIYDtCMLtEgKOBaaONOdHyZCTfzHdrJoWbuW '
app_secret = 'f0f1c03eff0f90ea299817d3db8f408f'
app_id = '204816748966658'

FacebookAdsApi.init(app_id, app_secret, access_token)

me = User(fbid='me')
my_accounts = list(me.get_ad_accounts())
print(my_accounts) 

[<AdAccount> {
    "account_id": "2262440287401635",
    "id": "act_2262440287401635"
}, <AdAccount> {
    "account_id": "252130562447858",
    "id": "act_252130562447858"
}, <AdAccount> {
    "account_id": "179572480898721",
    "id": "act_179572480898721"
}, <AdAccount> {
    "account_id": "1001912123901937",
    "id": "act_1001912123901937"
}, <AdAccount> {
    "account_id": "578146190393834",
    "id": "act_578146190393834"
}]


In [4]:
insightFields = [  
    AdsInsights.Field.campaign_name,
    AdsInsights.Field.campaign_id,
    AdsInsights.Field.adset_id,
    AdsInsights.Field.adset_name,
    AdsInsights.Field.ad_id,
    AdsInsights.Field.ad_name,
    AdsInsights.Field.impressions, # 노출
    AdsInsights.Field.clicks,
    AdsInsights.Field.cpm, # 1000 노출당 비용
    AdsInsights.Field.ctr, # 사람들이 광고를 보고 클릭(전체) 행동을 취한 비율
    AdsInsights.Field.cpp, # 사람들이 광고를 보고 클릭(전체) 행동을 취한 비율
    AdsInsights.Field.spend, # 광고비
    AdsInsights.Field.reach, # 도달
    AdsInsights.Field.purchase_roas,
    AdsInsights.Field.conversions,
    AdsInsights.Field.date_start,
    AdsInsights.Field.date_stop,
    AdsInsights.Field.objective, # 광고 목적
    AdsInsights.Field.actions, # purchase(구매수)
]

begin = datetime.date(2022, 12, 10)
end = datetime.date(2022, 12, 15)

# 전체 데이터 저장 할 변수
total_li = []
for day in tqdm_notebook(range((end - begin).days)):
    day = begin + datetime.timedelta(days=day) 
    day = day.strftime('%Y-%m-%d') 
    
    # 일단위 광고 데이터 받아오기
    # 캠페인, 광고셋, 광고 중 최소단위인 광고데이터를 성별과 나이로 구분
    params = {
        'time_range': {'since': day, 'until': day},
        'level': 'ad',
        'breakdowns': ['age', 'gender'],
    }
    
    # 광고계정 연결
    insights = AdAccount('act_252130562447858').get_insights(params=params, fields=insightFields)
    
 
    # 1일 단위의 광고 정보 
    for ins in insights: 
        purchase_roas = 0
        # 로하스 값만 가져오기
        if 'purchase_roas' in ins:
            purchase_roas = ins['purchase_roas'][0]['value'] 

        
        purchase = 0
        post_engagement = 0
        if 'actions' in ins:
            actions = ins['actions']
            purchase = list(filter(lambda x: x['action_type'] == 'purchase', ins['actions'] ))
            if len(purchase) > 0:
                purchase = purchase[0]['value']

            post_engagement = list(filter(lambda x: x['action_type'] == 'purchase', ins['actions'] ))
            if len(post_engagement) > 0:
                post_engagement = post_engagement[0]['value']
        
        cpp = -1
        if 'cpp' in ins:
            cpp = ins['cpp']

        cpm = -1
        if 'cpm' in ins:
            cpm = ins['cpm']

        ctr = 0
        if 'ctr' in ins:
            ctr = ins['ctr'] 

        row = {
            'objective': ins['objective'],
            'campaign_id':ins['campaign_id'],
            'adset_id': ins['adset_id'], 
            'ad_id':ins['ad_id'], 
            'ad_name':ins['ad_name'],
            'gender':ins['gender'], 
            'age':ins['age'],
            'date_start':ins['date_start'],
            'date_stop':ins['date_stop'],
            # 'date' : since, 
            'spend':ins['spend'], 
            'impressions':ins['impressions'], 
            'clicks':ins['clicks'],  
            'reach':ins['reach'], 
            'cpp':cpp, 
            'ctr':ctr, 
            'cpm':cpm, 
            'purchase_roas':purchase_roas, 
            'purchase':purchase,  
        }

        total_li.append(row) 
        
rdf = pd.DataFrame(total_li)

C:\Users\user\AppData\Local\Temp\ipykernel_11504\2365197128.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for day in tqdm_notebook(range((end - begin).days)):


  0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
rdf

,objective,campaign_id,adset_id,ad_id,ad_name,gender,age,date_start,date_stop,spend,impressions,clicks,reach,cpp,ctr,cpm,purchase_roas,purchase
0,OUTCOME_SALES,23851527615230055,23851527615290055,23851527677030055,할인_221024,male,25-34,2022-12-10,2022-12-10,103,6,0,6,17166.666667,0,17166.666667,0,0
1,OUTCOME_SALES,23851527615230055,23851527615290055,23851527677030055,할인_221024,female,35-44,2022-12-10,2022-12-10,40,1,0,1,40000,0,40000,0,0
2,OUTCOME_SALES,23851527615230055,23851527615290055,23851527677030055,할인_221024,male,35-44,2022-12-10,2022-12-10,485,13,0,12,40416.666667,0,37307.692308,0,0
3,OUTCOME_SALES,23851527615230055,23851527615290055,23851527677030055,할인_221024,female,45-54,2022-12-10,2022-12-10,7,1,0,1,7000,0,7000,0,0
4,OUTCOME_SALES,23851527615230055,23851527615290055,23851527677030055,할인_221024,male,45-54,2022-12-10,2022-12-10,44,2,0,2,22000,0,22000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1478,OUTCOME_SALES,23851985510390055,23851985510420055,23851985634910055,할인_221209,female,45-54,2022-12-14,2022-12-14,422,13,0,12,35166.666667,0,32461.538462,0,0
1479,OUTCOME_SALES,23851985510390055,23851985510420055,23851985634910055,할인_221209,male,45-54,2022-12-14,2022-12-14,585,12,0,8,73125,0,48750,0,0
1480,OUTCOME_SALES,23851985510390055,23851985510420055,23851985634910055,할인_221209,female,55-64,2022-12-14,2022-12-14,119,2,0,2,59500,0,59500,0,0
1481,OUTCOME_SALES,23851985510390055,23851985510420055,23851985634910055,할인_221209,male,55-64,2022-12-14,2022-12-14,79,2,0,2,39500,0,39500,0,0
